# Full run through for performing GWAS in Hail

> This notebook runs a basic GWAS in Hail after initially converting and performing quality control on 200k pVCF WES files. Overall, this notebook provides a complete standalone workflow of basic elements for performing a GWAS in Hail. Much of the code is based on the guidance accessible here: https://documentation.dnanexus.com/science/using-hail-to-analyze-genomic-data - but see also hyperlinks throughout notebook.


- runtime: 2hrs 
- recommended instance: mem1_ssd1_v2_x16 - **NB set up RAP instance using Spark with the "HAIL-VEP" feature (default Hail will fail) and two nodes**
- cost: <£0.80

This notebook depends on:
* **A Spark instance**
* **Notebook _G103_Retrieve_participant_data_for_Hail_GWAS.ipynb_**

Please note, before committing to performing large-scale workflows with Hail, a user must first consider any drawbacks associated with Hail (e.g. https://community.dnanexus.com/s/question/0D5t000004AflSiCAJ/hail-troubleshooting-for-ukb-data). These may include compression method employed on your target data (e.g. *zstd* compression, associated with UK Biobank WES BGEN formatted files, is not supported in Hail - in this case file conversion to *zlib* would be required). Further, as for all genomics projects, some consideration needs to be exercised regarding employed human genome build (i.e. GRCh37 vs. GRCh38). Finally, although Hail may expedite faster workflow performance, our own testing indicates Hail may not always store data as efficiently as the original format of the data (see last section of this notebook) - but see https://hail.is/docs/0.2/vds/index.html for Hail matrix storage solutions.

#### Initiate Spark and Hail

In [ ]:
# https://documentation.dnanexus.com/science/using-hail-to-analyze-genomic-data

from pyspark.sql import SparkSession
import hail as hl

builder = (
    SparkSession
    .builder
    .enableHiveSupport())
spark = builder.getOrCreate()
hl.init(sc=spark.sparkContext)

#### 1. Grab data and render as Hail Matrix
We'll use WES (field 23156) and use chr20 as it's one of the smallest chromosomes:

In [ ]:
%%bash
# list files for field 23156
dx ls -l "Bulk/Exome sequences_Previous exome releases/Population level exome OQFE variants, pVCF format - interim 200k release/ukb23156_c20_*vcf.gz"

In [ ]:
# Import genomic data as a Hail matrix (MT)
file_url = "file:///mnt/project/Bulk/Exome sequences_Previous exome releases/Population level exome OQFE variants, pVCF format - interim 200k release/ukb23156_c20_*.vcf.gz"
vcf = hl.import_vcf(file_url, reference_genome='GRCh38', force_bgz=True, array_elements_required=False)

In [ ]:
# Describe the data (this step may take a while and can be skipped)
print(f"Num partitions: {vcf.n_partitions()}")
vcf.describe()

In [ ]:
# Define database and MT names
# Note: It is recommended to only use lowercase letters for the database name.
# If uppercase lettering is used, the database name will be lowercased when creating the database.
db_name = "hail_gwas_db2" # previosuly named: "database_name"
mt_name = "c20_geno.mt" # needs to be new name if creating new DB (don't try to overwrite)

In [ ]:
# Create Apollo database (dnax://) on the DNAnexus platform (DNAX).
stmt = f"CREATE DATABASE IF NOT EXISTS {db_name} LOCATION 'dnax://'"
print(stmt)
spark.sql(stmt).show()

In [ ]:
# Store MT in DNAX (#NB may take 1hr for chr20)
import dxpy

# Find database ID of newly created database using dxpy method
db_uri = dxpy.find_one_data_object(name=f"{db_name}", classname="database")['id']
url = f"dnax://{db_uri}/{mt_name}" # Note: the dnax url must follow this format to properly save MT to DNAX

# Before this step, the Hail MatrixTable is just an object in memory. To persist it and be able to access 
# it later, the notebook needs to write it into a persistent filesystem (in this case DNAX).
# See https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.write for additional documentation.
vcf.write(url) # Note: output should describe size of MT (i.e. number of rows, columns, partitions) 


#### 2. Let's perform some QC on the data
see: https://github.com/dnanexus/OpenBio/blob/master/hail_tutorial/locus_qc.ipynb & https://github.com/dnanexus/OpenBio/blob/master/hail_tutorial/sample_qc.ipynb

In [ ]:
db_uri = dxpy.find_one_data_object(name=f"{db_name}", classname="database")['id']
url = f"dnax://{db_uri}/{mt_name}" 

In [ ]:
# read MT
mt = hl.read_matrix_table(url)

In [ ]:
# Run Hail's variant-level QC
qc_mt = hl.variant_qc(mt)

In [ ]:
# View structure of MT after QC
qc_mt.describe()

In [ ]:
# Create Hail Table from MT
qc_tb = qc_mt.rows()

# Define table name
tb_name = "variant_qc.ht"

In [ ]:
# Create database in DNAX - only need running (i.e. uncommenting) if you are jumping in at this point

#stmt = f"CREATE DATABASE IF NOT EXISTS {db_name} LOCATION 'dnax://'"
#print(stmt)
#spark.sql(stmt).show()

In [ ]:
# Store Table in DNAX

# find database ID of newly created database using a dxpy method
db_uri = dxpy.find_one_data_object(name=f"{db_name}", classname="database")['id']
url = f"dnax://{db_uri}/{tb_name}"

qc_tb.write(url) # Note: output should describe size of Table (i.e. number of rows, partitions)

In [ ]:
# Run Hail sample-level QC
qc_mt = hl.sample_qc(mt)

In [ ]:
# Create Hail Table from MT
qc_tb = qc_mt.cols()

# Define table name
tb_name = "sample_qc.ht"

In [ ]:
# Create database in DNAX - only need running (i.e. uncommenting) if you are jumping in at this point

#stmt = f"CREATE DATABASE IF NOT EXISTS {db_name} LOCATION 'dnax://'"
#print(stmt)
#spark.sql(stmt).show()

In [ ]:
# Store Table in DNAX

# find database ID of newly created database using a dxpy method
db_uri = dxpy.find_one_data_object(name=f"{db_name}", classname="database")['id']
url = f"dnax://{db_uri}/{tb_name}"

qc_tb.write(url) # Note: output should describe size of Table (i.e. number of rows, partitions)

#### 3. Apply preliminary QC for GWAS 

In [ ]:
# If starting from here you need to identify Hail matrix to be used
# This is the name of your main Hail matrix for the genomic data; variable: *db_name*
import dxpy

db_uri = dxpy.find_one_data_object(name=f"{db_name}", classname="database")['id']
url = f"dnax://{db_uri}/{mt_name}" 

mt = hl.read_matrix_table(url)

In [ ]:
# Use phenotypic table made in notebook "G103_Retrieve_participant_data_for_Hail_GWAS.ipynb" that should have been previously run
# Supposing smoking_bool.csv is the name of the table you created in the pheno folder from the previous notebook. 
pheno_table = hl.import_table("file:///mnt/project/pheno/smoking_bool.csv",
                              delimiter=',',
                              impute=True,
                              key='eid')

In [ ]:
# QUICK HACK TO CONVERT EIDs TO STRINGS
pheno_table = pheno_table.annotate(eid2 = hl.str(pheno_table.eid))
pheno_table = pheno_table.key_by('eid2')
pheno_table = pheno_table.drop('eid')

In [ ]:
pheno_table.describe()

In [ ]:
# Annotate the MT with pheno Table by matching the MT's column key ('s': samples) with the pheno Table's key ('sample_id')
phenogeno_mt = mt.annotate_cols(**pheno_table[mt.s])

In [ ]:
phenogeno_mt.describe()

In [ ]:
# Define sample QC Table url
locus_qc_url = f"dnax://{db_uri}/variant_qc.ht"

# Read sample QC Table
pre_locus_qc_tb = hl.read_table(locus_qc_url)


In [ ]:
# Filter QC Table using expressions
# Note: Viewing the structure of the locus QC table in from the cell above 
# shows us that the "AF", "p_value_hwe", and "call_rate" fields are within
# the "variant_qc" struct field.

locus_qc_tb = pre_locus_qc_tb.filter(
    (pre_locus_qc_tb["variant_qc"]["AF"][0] >= 0.001) & 
    (pre_locus_qc_tb["variant_qc"]["AF"][0] <= 0.999) & 
    (pre_locus_qc_tb["variant_qc"]["p_value_hwe"] >= 1e-10) & 
    (pre_locus_qc_tb["variant_qc"]["call_rate"] >= 0.9))

In [ ]:
# View number of loci in QC Table before and after filtering
#
# Note: running this cell can be computationally expensive and take
# longer for bigger datasets (this cell can be commented out).

print(f"Num loci before filtering: {pre_locus_qc_tb.count()}")
print(f"Num loci after filtering: {locus_qc_tb.count()}")

In [ ]:
# Define sample QC Table url
sample_qc_url = f"dnax://{db_uri}/sample_qc.ht"

# Read sample QC Table
pre_sample_qc_tb = hl.read_table(sample_qc_url)

# View structure of sample QC Table
pre_sample_qc_tb.describe()

In [ ]:
#Let's filter for samples that have a call rate greater or equal to 0.99

# Filter sample QC Table using expressions
# Note: Viewing the structure of the sample QC table in from the cell above 
# shows us that the "call_rate" field is within the "sample_qc" struct field

sample_qc_tb = pre_sample_qc_tb.filter(pre_sample_qc_tb["sample_qc"]["call_rate"] >= 0.99) 

In [ ]:
# View number of samples in QC Table before and after filtering
#
# Note: running this cell can be computationally expensive and take
# longer for bigger datasets (this cell can be commented out).

print(f"Num samples before filtering: {pre_sample_qc_tb.count()}")
print(f"Num samples after filtering: {sample_qc_tb.count()}")

In [ ]:
# Filter the MT using the locus QC Table
qc_mt = phenogeno_mt.semi_join_rows(locus_qc_tb)

# Filter the MT using the sample QC Table
qc_mt = qc_mt.semi_join_cols(sample_qc_tb)

# View MT after QC filters
qc_mt.describe()

#### 4. Perform GWAS
see: https://github.com/dnanexus/OpenBio/blob/master/hail_tutorial/gwas.ipynb

In [ ]:
## Run GWAS
# Additional documentation: https://hail.is/docs/0.2/methods/stats.html#hail.methods.logistic_regression_rows

# Run Hail's logistic regression method:
gwas = hl.logistic_regression_rows(test="firth",
                                   y=qc_mt.smoking, # the column field of the pheno trait we are looking at ('smoking') - this should be changed if you are interesed in another trait.
                                   x=qc_mt.GT.n_alt_alleles(), # n_alt_alleles() returns the count of non-reference alleles
                                   covariates=[1.0])

In [ ]:
# View structure of GWAS results Table
gwas.describe()

In [ ]:
# Save GWAS results Table in Apollo Database
# Define database and table name

# Note: It is recommended to only use lowercase letters for the database name.
# If uppercase lettering is used, the database name will be lowercased when creating the database.
tb_name = "gwas.ht"


In [ ]:
# Create database in DNAX - only need running (i.e. uncommenting) if you are jumping in at this point

#stmt = f"CREATE DATABASE IF NOT EXISTS {db_name} LOCATION 'dnax://'"
#print(stmt)
#spark.sql(stmt).show()


In [ ]:
# Store Table in DNAX

import dxpy

# find database ID of newly created database using a dxpy method
db_uri = dxpy.find_one_data_object(name=f"{db_name}", classname="database")['id']
url = f"dnax://{db_uri}/{tb_name}"

gwas.write(url) # Note: output should describe size of Table (i.e. number of rows, partitions)

#### 5. Visualize GWAS results with Hail: https://github.com/dnanexus/OpenBio/blob/master/hail_tutorial/gwas_vis.ipynb

In [ ]:
# Read GWAS results table
# Define GWAS results Table url
db_uri = dxpy.find_one_data_object(name=f"{db_name}", classname="database")['id']
tb_url = f"dnax://{db_uri}/gwas.ht"
# Read GWAS results Table

gwas_tb = hl.read_table(tb_url)
# View structure of Table

gwas_tb.describe()

In [22]:
#Import Bokeh to visualize plots
#Bokeh is a Python library that is included in this JupyterLab environment- which makes it easy for us to import!

from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [23]:
#Create Q-Q plot
#Additional documentation: https://hail.is/docs/0.2/plot.html#hail.plot.qq
qq_plot = hl.plot.qq(gwas_tb.p_value)
show(qq_plot)

2024-01-12 15:02:08.419 Hail: INFO: Ordering unsorted dataset with network shuffle

2024-01-12 15:02:19.266 Hail: INFO: Ordering unsorted dataset with network shuffle


Deviation of the QQ plot from 1:1 slope suggests that either data wrangling of the phenotypic data or employment of a different GWAS model may improve predictive performance. NB as this is not an in depth quide into building an optimised GWAS model it is the user's responsibility to research the most appropriate methodology.

In [24]:
#Create Manhattan plot
#Additional documentation: https://hail.is/docs/0.2/plot.html#hail.plot.manhattan
manhattan_plot = hl.plot.manhattan(gwas_tb.p_value)
show(manhattan_plot)

The manhattan indicates some potential significant alleles.

#### 6. Data storage comparison
Compare size of Hail database versus equivalent joint-call pVCFs.

In [2]:
%%bash
echo Hail DB size: $(dx list database files database-XXX --recurse | grep c20_geno | awk '{sum+=$3} END {print sum}') bytes # where XXX is the unique identifier number of the database

Hail DB size: 2.52076e+11 bytes


Hail DB is 252Gb

In [3]:
%%bash
echo Joint-call pVCFs size: $(dx find data --property field_id=23156 | grep c20 | grep -v tbi | awk '{sum+=$4} END {print sum}') GB

Joint-call pVCFs size: 177.3 GB


The Hail database comprising the genomic data is *ca*. 1.42x the size of the original VCFs. This should be considered when evaluating the value of Hail. See also: https://hail.is/docs/0.2/vds/index.html 